In [ ]:
import os
import glob
import pandas as pd

# Set the directory path where the files are located
directory = "d:\\DataExport\\"

# Get a list of all CSV files in the directory
file_list = glob.glob(os.path.join(directory, "*.csv"))

# Create an empty DataFrame to store the merged data
merged_data = pd.DataFrame()

# Iterate over each file and merge the data
for file in file_list:
    df = pd.read_csv(file)
    merged_data = pd.concat([merged_data, df])

# Write the merged data to a new CSV file
merged_data.to_csv("stock_data.csv", index=False)


In [91]:
import pandas as pd
import configparser

# Read the value from the config.ini file
config = configparser.ConfigParser()
config.read('config.ini')
try:
    file_path = config.get('data', 'file_path1')
    # Read the data from the csv file
    df = pd.read_csv(file_path, engine='pyarrow')

except FileNotFoundError as e:
    file_path = config.get('data', 'file_path2')
    # Read the data from the csv file
    df = pd.read_csv(file_path, engine='pyarrow')
except:
    print('Error reading the file path from the config file')

print(df.head())

    Symbol    Date    Open    High     Low   Close  Volume
0  VN30F1M  220810  1272.3  1273.7  1272.1  1272.8    7684
1  VN30F1M  220810  1272.7  1273.7  1271.0  1271.1   18341
2  VN30F1M  220810  1271.1  1271.2  1271.1  1271.2     121
3  VN30F1M  220810  1271.2  1271.2  1269.4  1270.4   21363
4  VN30F1M  220810  1270.3  1272.9  1268.7  1270.7   52325


In [92]:
# Drop the rows with missing values
df.dropna(inplace=True)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1504 entries, 0 to 1503
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Symbol  1504 non-null   object 
 1   Date    1504 non-null   int64  
 2   Open    1504 non-null   float64
 3   High    1504 non-null   float64
 4   Low     1504 non-null   float64
 5   Close   1504 non-null   float64
 6   Volume  1504 non-null   int64  
dtypes: float64(4), int64(2), object(1)
memory usage: 82.4+ KB
None


In [93]:
import pandas as pd
import numpy as np

# Assuming your dataframe is named "df"
# Convert the "Date" column to datetime format
df['Date'] = pd.to_datetime(df['Date'], format='%y%m%d')

# Map the day of the week to corresponding labels
day_labels = ['Mo', 'Tu', 'We', 'Th', 'Fr', 'Sa', 'Su']
df['DoW'] = df['Date'].dt.dayofweek.map(lambda x: day_labels[x])

# Check if the date is the 3rd Thursday of the month and label it as "Th-expired"
DoWExp = config.get('derivatives', 'DoWExp')

df['DoW'] = np.where((df['Date'].dt.weekday == 3) & (df['Date'].dt.day <= 21) & (df['Date'].dt.day >= 15), DoWExp + '-exp', df['DoW'])

# Revert the "Date" column back to the original format
df['Date'] = df['Date'].dt.strftime('%y%m%d')

# Print the updated dataframe
print(df)


       Symbol    Date    Open    High     Low   Close  Volume DoW
0     VN30F1M  220810  1272.3  1273.7  1272.1  1272.8    7684  We
1     VN30F1M  220810  1272.7  1273.7  1271.0  1271.1   18341  We
2     VN30F1M  220810  1271.1  1271.2  1271.1  1271.2     121  We
3     VN30F1M  220810  1271.2  1271.2  1269.4  1270.4   21363  We
4     VN30F1M  220810  1270.3  1272.9  1268.7  1270.7   52325  We
...       ...     ...     ...     ...     ...     ...     ...  ..
1499  VN30F1M  230630  1117.1  1117.9  1114.6  1115.3   23891  Fr
1500  VN30F1M  230630  1115.3  1115.5  1115.3  1115.5      19  Fr
1501  VN30F1M  230630  1115.2  1118.9  1115.2  1118.6   25428  Fr
1502  VN30F1M  230630  1118.6  1119.9  1115.1  1119.0   56744  Fr
1503  VN30F1M  230630  1119.1  1119.1  1117.3  1117.3    8736  Fr

[1504 rows x 8 columns]


In [94]:
import numpy as np

# Create the FO column
df['Adj_Open'] = np.where(df['Open'] > df['Close'].shift(1), 'OU', 'OD')

# Create the FC column
df['Adj_Close'] = np.where(df['Close'] > df['Close'].shift(1), 'CU', 'CD')

# Calculate the point of High from Previous Close
df['HPC'] = (df['High'] - df['Close'].shift(1))

# Calculate the point of Low from Previous Close
df['LPC'] = (df['Low'] - df['Close'].shift(1))

# Calculate the point of High from Open
df['HFO'] = (df['High'] - df['Open'])

# Calculate the point of Low from Open
df['LFO'] = (df['Low'] - df['Open'])

# Calculate the point of High from Close
df['HFC'] = (df['High'] - df['Close'])

# Calculate the point of Low from Close
df['LFC'] = (df['Low'] - df['Close'])

In [95]:
test_df = df[df['DoW'] == 'Th-exp']

# Count number of CD and CU in test_df
print(test_df['Adj_Close'].value_counts())


# Count number of OU and OD in test_df
print(test_df['Adj_Open'].value_counts())



Adj_Close
CD    41
CU    36
Name: count, dtype: int64
Adj_Open
OD    48
OU    29
Name: count, dtype: int64


In [96]:
# Count number of CD and CU in df, excluding Th-exp
df_excl_th = df[df['DoW'] != (DoWExp + '-exp')]
print(df_excl_th['Adj_Close'].value_counts())



Adj_Close
CD    795
CU    632
Name: count, dtype: int64


In [97]:
import pandas as pd

# Assuming your dataframe is named "df"

# Define the categories
categories = ['CU', 'CD']

# Create a pivot table to calculate the count
pivot_df = pd.pivot_table(df, index='DoW', values='Adj_Open', columns='Adj_Close', aggfunc='count', fill_value=0)

# Calculate the total count
pivot_df['Total'] = pivot_df.sum(axis=1)

# Calculate the percentages
for category in categories:
    pivot_df[f'%{category}'] = pivot_df[category] / pivot_df['Total']

# Reorder the columns
columns = categories + [f'%{category}' for category in categories] + ['Total']
pivot_df = pivot_df.reindex(columns=columns)

# Reindex the rows to match the desired order
day_labels = ['Su', 'Mo', 'Tu', 'We', 'Th',(DoWExp + '-exp'), 'Fr', 'Sa']
pivot_df = pivot_df.reindex(day_labels)

# Drop Total column
pivot_df.drop('Total', axis=1, inplace=True)

pivot_df.dropna(inplace=True)

pivot_df

Adj_Close,CU,CD,%CU,%CD
DoW,,,,
Mo,115.0,172.0,0.400697,0.599303
Tu,149.0,152.0,0.495017,0.504983
We,137.0,170.0,0.446254,0.553746
Th,97.0,134.0,0.419913,0.580087
Th-exp,36.0,41.0,0.467532,0.532468
Fr,134.0,167.0,0.445183,0.554817


In [98]:
import pandas as pd

# Assuming your dataframe is named "df"

# Define the categories
categories = ['OU', 'OD']

# Create a pivot table to calculate the count
pivot_df_1 = pd.pivot_table(df, index='DoW', values='Adj_Close', columns='Adj_Open', aggfunc='count', fill_value=0)

# Calculate the total count
pivot_df_1['Total'] = pivot_df_1.sum(axis=1)

# Calculate the percentages
for category in categories:
    pivot_df_1[f'%{category}'] = pivot_df_1[category] / pivot_df_1['Total']

# Reorder the columns
columns = categories + [f'%{category}' for category in categories] + ['Total']
pivot_df_1 = pivot_df_1.reindex(columns=columns)

# Reindex the rows to match the desired order
day_labels = ['Su', 'Mo', 'Tu', 'We', 'Th',(DoWExp + '-exp'), 'Fr', 'Sa']
pivot_df_1 = pivot_df_1.reindex(day_labels)

# Drop Total column
pivot_df_1.drop('Total', axis=1, inplace=True)

# Drop the rows with missing values
pivot_df_1.dropna(inplace=True)

pivot_df_1

Adj_Open,OU,OD,%OU,%OD
DoW,,,,
Mo,113.0,174.0,0.393728,0.606272
Tu,109.0,192.0,0.362126,0.637874
We,116.0,191.0,0.377850,0.622150
Th,88.0,143.0,0.380952,0.619048
Th-exp,29.0,48.0,0.376623,0.623377
Fr,117.0,184.0,0.388704,0.611296


In [99]:
df

,Symbol,Date,Open,High,Low,Close,Volume,DoW,Adj_Open,Adj_Close,HPC,LPC,HFO,LFO,HFC,LFC
0,VN30F1M,220810,1272.3,1273.7,1272.1,1272.8,7684,We,OD,CD,NaN,NaN,1.4,-0.2,0.9,-0.7
1,VN30F1M,220810,1272.7,1273.7,1271.0,1271.1,18341,We,OD,CD,0.9,-1.8,1.0,-1.7,2.6,-0.1
2,VN30F1M,220810,1271.1,1271.2,1271.1,1271.2,121,We,OD,CU,0.1,0.0,0.1,0.0,0.0,-0.1
3,VN30F1M,220810,1271.2,1271.2,1269.4,1270.4,21363,We,OD,CD,0.0,-1.8,0.0,-1.8,0.8,-1.0
4,VN30F1M,220810,1270.3,1272.9,1268.7,1270.7,52325,We,OD,CU,2.5,-1.7,2.6,-1.6,2.2,-2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1499,VN30F1M,230630,1117.1,1117.9,1114.6,1115.3,23891,Fr,OU,CD,0.9,-2.4,0.8,-2.5,2.6,-0.7
1500,VN30F1M,230630,1115.3,1115.5,1115.3,1115.5,19,Fr,OD,CU,0.2,0.0,0.2,0.0,0.0,-0.2
1501,VN30F1M,230630,1115.2,1118.9,1115.2,1118.6,25428,Fr,OD,CU,3.4,-0.3,3.7,0.0,0.3,-3.4
1502,VN30F1M,230630,1118.6,1119.9,1115.1,1119.0,56744,Fr,OD,CU,1.3,-3.5,1.3,-3.5,0.9,-3.9


In [100]:
import pandas as pd
import numpy as np

# Create the points dataframe
points_df = pd.DataFrame({'Points': range(1, 15)})

# Calculate the percentage of points >= HFO
points_df['%HFO'] = points_df['Points'].apply(lambda x: (df['HFO'] >= x).mean() * 100)

# Calculate the percentage of points >= abs(LFO)
points_df['%LFO'] = points_df['Points'].apply(lambda x: (np.abs(df['LFO']) >= x).mean() * 100)

# Calculate the percentage of points >= HFC
points_df['%HFC'] = points_df['Points'].apply(lambda x: (df['HFC'] >= x).mean() * 100)

# calculate the percentage of points >= abs(LFC)
points_df['%LFC'] = points_df['Points'].apply(lambda x: (np.abs(df['LFC']) >= x).mean() * 100)

# Calculate the percentage of points >= HPC
points_df['%HPC'] = points_df['Points'].apply(lambda x: (df['HPC'] >= x).mean() * 100)

# Calculate the percentage of points >= abs(LPC)
points_df['%LPC'] = points_df['Points'].apply(lambda x: (np.abs(df['LPC']) >= x).mean() * 100)
# Print the resulting dataframe
print(points_df)


    Points       %HFO       %LFO       %HFC       %LFC       %HPC       %LPC
0        1  57.646277  58.843085  61.835106  58.510638  57.047872  62.034574
1        2  42.553191  43.617021  44.813830  41.755319  43.417553  47.074468
2        3  33.311170  33.510638  34.441489  31.449468  35.172872  36.901596
3        4  25.997340  27.127660  26.462766  23.869681  28.058511  30.252660
4        5  20.811170  21.143617  20.877660  18.949468  23.138298  23.936170
5        6  16.821809  16.622340  17.287234  14.827128  18.750000  19.747340
6        7  13.497340  12.765957  14.295213  11.436170  15.625000  15.359043
7        8  10.505319  10.704787  11.569149   9.441489  12.167553  13.164894
8        9   8.710106   8.577128   9.441489   8.377660  10.172872  10.970745
9       10   7.180851   6.648936   7.513298   6.848404   8.311170   8.909574
10      11   5.851064   5.718085   6.582447   5.452128   7.114362   7.646277
11      12   4.720745   4.853723   5.452128   4.587766   5.984043   6.316489

In [101]:
import pandas as pd

# Method 1: Buying at the close and selling at the open the next day
df_method1 = df.copy()
df_method1['Entry'] = df_method1['Close'].shift()
df_method1['Exit'] = df_method1['Open']
df_method1['Points'] = df_method1['Exit'] - df_method1['Entry']
cumulative_points_method1 = df_method1['Points'].cumsum().iloc[-1]

# Method 2: Shorting at the open plus 2 points and covering at the close
df_method2 = df.copy()
df_method2['Entry'] = df_method2['Open'] + 2
df_method2['Exit'] = df_method2['Close']
df_method2['Points'] = df_method2['Exit'] - df_method2['Entry']
cumulative_points_method2 = df_method2['Points'].cumsum().iloc[-1]

# Method 3: Buying at the open minus 2 points and selling at the close
df_method3 = df.copy()
df_method3['Entry'] = df_method3['Open'] - 2
df_method3['Exit'] = df_method3['Close']
df_method3['Points'] = df_method3['Exit'] - df_method3['Entry']
cumulative_points_method3 = df_method3['Points'].cumsum().iloc[-1]

# Compare the cumulative points for each method
method_points = {
    'Method 1': cumulative_points_method1,
    'Method 2': cumulative_points_method2,
    'Method 3': cumulative_points_method3
}

# Find the method with the most points
best_method = max(method_points, key=method_points.get)
best_points = method_points[best_method]

# Print the results
for method, points in method_points.items():
    print(f"{method}: Cumulative Points = {points}")

print(f"\nBest Method: {best_method}")
print(f"Cumulative Points: {best_points}")


Method 1: Cumulative Points = 62.70000000000175
Method 2: Cumulative Points = -3225.7000000000025
Method 3: Cumulative Points = 2790.2999999999993

Best Method: Method 3
Cumulative Points: 2790.2999999999993


In [102]:
df

,Symbol,Date,Open,High,Low,Close,Volume,DoW,Adj_Open,Adj_Close,HPC,LPC,HFO,LFO,HFC,LFC
0,VN30F1M,220810,1272.3,1273.7,1272.1,1272.8,7684,We,OD,CD,NaN,NaN,1.4,-0.2,0.9,-0.7
1,VN30F1M,220810,1272.7,1273.7,1271.0,1271.1,18341,We,OD,CD,0.9,-1.8,1.0,-1.7,2.6,-0.1
2,VN30F1M,220810,1271.1,1271.2,1271.1,1271.2,121,We,OD,CU,0.1,0.0,0.1,0.0,0.0,-0.1
3,VN30F1M,220810,1271.2,1271.2,1269.4,1270.4,21363,We,OD,CD,0.0,-1.8,0.0,-1.8,0.8,-1.0
4,VN30F1M,220810,1270.3,1272.9,1268.7,1270.7,52325,We,OD,CU,2.5,-1.7,2.6,-1.6,2.2,-2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1499,VN30F1M,230630,1117.1,1117.9,1114.6,1115.3,23891,Fr,OU,CD,0.9,-2.4,0.8,-2.5,2.6,-0.7
1500,VN30F1M,230630,1115.3,1115.5,1115.3,1115.5,19,Fr,OD,CU,0.2,0.0,0.2,0.0,0.0,-0.2
1501,VN30F1M,230630,1115.2,1118.9,1115.2,1118.6,25428,Fr,OD,CU,3.4,-0.3,3.7,0.0,0.3,-3.4
1502,VN30F1M,230630,1118.6,1119.9,1115.1,1119.0,56744,Fr,OD,CU,1.3,-3.5,1.3,-3.5,0.9,-3.9


In [103]:
import pandas as pd

# Convert the Date column to string format
df['Date'] = df['Date'].astype(str)

# Filter the dataframe for the desired date range (2022 and 2023)
df_filtered = df[(df['Date'] >= '220101') & (df['Date'] <= '231231')]

# Method 1: Buying at the close and selling at the open the next day
df_method1 = df_filtered.copy()
df_method1['Entry'] = df_method1['Close'].shift()
df_method1['Exit'] = df_method1['Open']
df_method1['Points'] = df_method1['Exit'] - df_method1['Entry']

# Calculate cumulative points for Method 1
if len(df_method1) > 0:
    cumulative_points_method1 = df_method1['Points'].cumsum().iloc[-1]
else:
    cumulative_points_method1 = 0

# Method 2: Shorting at the open plus 2 points and covering at the close
df_method2 = df_filtered.copy()
df_method2['Entry'] = df_method2['Open'] + 2
df_method2['Exit'] = df_method2['Close']
df_method2['Points'] = df_method2['Exit'] - df_method2['Entry']

# Calculate cumulative points for Method 2
if len(df_method2) > 0:
    cumulative_points_method2 = df_method2['Points'].cumsum().iloc[-1]
else:
    cumulative_points_method2 = 0

# Method 3: Buying at the open minus 2 points and selling at the close
df_method3 = df_filtered.copy()
df_method3['Entry'] = df_method3['Open'] - 2
df_method3['Exit'] = df_method3['Close']
df_method3['Points'] = df_method3['Exit'] - df_method3['Entry']

# Calculate cumulative points for Method 3
if len(df_method3) > 0:
    cumulative_points_method3 = df_method3['Points'].cumsum().iloc[-1]
else:
    cumulative_points_method3 = 0

# Compare the cumulative points for each method
method_points = {
    'Method 1': cumulative_points_method1,
    'Method 2': cumulative_points_method2,
    'Method 3': cumulative_points_method3
}

# Find the method with the most points
best_method = max(method_points, key=method_points.get)
best_points = method_points[best_method]

# Print the results
for method, points in method_points.items():
    print(f"{method}: Cumulative Points = {points}")

print(f"\nBest Method: {best_method}")
print(f"Cumulative Points: {best_points}")


Method 1: Cumulative Points = 62.70000000000175
Method 2: Cumulative Points = -3225.7000000000025
Method 3: Cumulative Points = 2790.2999999999993

Best Method: Method 3
Cumulative Points: 2790.2999999999993


In [104]:
### METHOD 3
import pandas as pd

# Convert the Date column to datetime format
df['Date'] = pd.to_datetime(df['Date'], format='%y%m%d')

# Filter the dataframe for the desired date range (2022 and 2023)
df_filtered = df[(df['Date'].dt.year.isin([2022, 2023]))]

# Method 3: Buying at the open minus 2 points and selling at the close
df_method3 = df_filtered.copy()
df_method3['Entry'] = df_method3['Open'] - 3
df_method3['Exit'] = df_method3['Close']
df_method3['Points'] = df_method3['Exit'] - df_method3['Entry']

# Calculate cumulative points for Method 3
cumulative_points_method3 = df_method3['Points'].cumsum()

# Group the data by month and year and calculate the points earned for each month and year
df_method3['Month'] = df_method3['Date'].dt.month
df_method3['Year'] = df_method3['Date'].dt.year
monthly_points = df_method3.groupby(['Year', 'Month'])['Points'].sum()

# Print the points earned each month and year
print("Points Earned Each Month and Year:")
print(monthly_points)


Points Earned Each Month and Year:
Year  Month
2022  8        378.3
      9        322.9
      10       302.6
      11       476.9
      12       406.7
2023  1        419.3
      2        312.5
      3        390.3
      4        352.9
      5        420.3
      6        511.6
Name: Points, dtype: float64


In [105]:
# Conver Date in df format YYYY-MM-DD to yymmdd
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d').dt.strftime('%y%m%d')

df

,Symbol,Date,Open,High,Low,Close,Volume,DoW,Adj_Open,Adj_Close,HPC,LPC,HFO,LFO,HFC,LFC
0,VN30F1M,220810,1272.3,1273.7,1272.1,1272.8,7684,We,OD,CD,NaN,NaN,1.4,-0.2,0.9,-0.7
1,VN30F1M,220810,1272.7,1273.7,1271.0,1271.1,18341,We,OD,CD,0.9,-1.8,1.0,-1.7,2.6,-0.1
2,VN30F1M,220810,1271.1,1271.2,1271.1,1271.2,121,We,OD,CU,0.1,0.0,0.1,0.0,0.0,-0.1
3,VN30F1M,220810,1271.2,1271.2,1269.4,1270.4,21363,We,OD,CD,0.0,-1.8,0.0,-1.8,0.8,-1.0
4,VN30F1M,220810,1270.3,1272.9,1268.7,1270.7,52325,We,OD,CU,2.5,-1.7,2.6,-1.6,2.2,-2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1499,VN30F1M,230630,1117.1,1117.9,1114.6,1115.3,23891,Fr,OU,CD,0.9,-2.4,0.8,-2.5,2.6,-0.7
1500,VN30F1M,230630,1115.3,1115.5,1115.3,1115.5,19,Fr,OD,CU,0.2,0.0,0.2,0.0,0.0,-0.2
1501,VN30F1M,230630,1115.2,1118.9,1115.2,1118.6,25428,Fr,OD,CU,3.4,-0.3,3.7,0.0,0.3,-3.4
1502,VN30F1M,230630,1118.6,1119.9,1115.1,1119.0,56744,Fr,OD,CU,1.3,-3.5,1.3,-3.5,0.9,-3.9


In [106]:
# METHOD TRADING USING MA 10

import pandas as pd

# Calculate MA10
df['MA10'] = df['Close'].rolling(window=10).mean()

# Define trading signals based on MA10
df['Signal'] = 0
df.loc[df['Close'] > df['MA10'], 'Signal'] = 1
df.loc[df['Close'] <= df['MA10'], 'Signal'] = -1

# Calculate trading returns
df['Returns'] = df['Close'].pct_change() * df['Signal'].shift()

# Calculate cumulative returns
df['Cumulative Returns'] = (1 + df['Returns']).cumprod()

# Evaluate trading performance
total_return = df['Cumulative Returns'].iloc[-1] - 1
annualized_return = (1 + total_return) ** (252 / len(df)) - 1
max_drawdown = 1 - df['Cumulative Returns'] / df['Cumulative Returns'].cummax()

print("Total Return:", total_return)
print("Annualized Return:", annualized_return)
print("Max Drawdown:", max_drawdown.min())


Total Return: 0.028830629696291465
Annualized Return: 0.0047736965685287736
Max Drawdown: 0.0


In [107]:
import pandas as pd

# Convert the Date column to string format
df['Date'] = df['Date'].astype(str)

# Filter the dataframe for the desired date range (2022 and 2023)
df_filtered = df[(df['Date'] >= '220101') & (df['Date'] <= '231231')]

# Calculate MA10
df_filtered['MA10'] = df_filtered['Close'].rolling(window=10).mean()

# Define trading signals based on MA10
df_filtered['Signal'] = 0
df_filtered.loc[df_filtered['Close'] > df_filtered['MA10'], 'Signal'] = 1
df_filtered.loc[df_filtered['Close'] <= df_filtered['MA10'], 'Signal'] = -1

# Calculate trading returns
df_filtered['Returns'] = df_filtered['Close'].pct_change() * df_filtered['Signal'].shift()

# Calculate cumulative returns
df_filtered['Cumulative Returns'] = (1 + df_filtered['Returns']).cumprod()

# Calculate points earned per month
df_filtered['YearMonth'] = pd.to_datetime(df_filtered['Date'], format='%y%m%d').dt.to_period('M')
monthly_points = df_filtered.groupby('YearMonth')['Returns'].sum()

print(monthly_points)


YearMonth
2022-08    0.057579
2022-09    0.038213
2022-10    0.080468
2022-11   -0.002166
2022-12   -0.050130
2023-01    0.071705
2023-02   -0.045427
2023-03   -0.070831
2023-04   -0.034462
2023-05    0.001705
2023-06    0.011470
Freq: M, Name: Returns, dtype: float64


In [108]:
## OPTIMIZE MA TRADING

import pandas as pd

# Define the range of MA values to test
ma_range = range(5, 51, 1)

# Initialize variables to store the best MA value and corresponding performance metrics
best_ma = None
best_annualized_return = float('-inf')
best_max_drawdown = float('inf')

# Iterate over the MA values
for ma in ma_range:
    # Calculate MA
    df['MA'] = df['Close'].rolling(window=ma).mean()

    # Define trading signals based on MA
    df['Signal'] = 0
    df.loc[df['Close'] > df['MA'], 'Signal'] = 1
    df.loc[df['Close'] <= df['MA'], 'Signal'] = -1

    # Calculate trading returns
    df['Returns'] = df['Close'].pct_change() * df['Signal'].shift()

    # Calculate cumulative returns
    df['Cumulative Returns'] = (1 + df['Returns']).cumprod()

    # Evaluate trading performance
    total_return = df['Cumulative Returns'].iloc[-1] - 1
    annualized_return = (1 + total_return) ** (252 / len(df)) - 1
    max_drawdown = 1 - df['Cumulative Returns'] / df['Cumulative Returns'].cummax()

    # Update the best performance metrics and MA value if necessary
    if annualized_return > best_annualized_return:
        best_ma = ma
        best_annualized_return = annualized_return
        best_max_drawdown = max_drawdown.min()

# Print the results
print("Best MA:", best_ma)
print("Best Annualized Return:", best_annualized_return)
print("Best Max Drawdown:", best_max_drawdown)


Best MA: 5
Best Annualized Return: 0.04584192034099144
Best Max Drawdown: 0.0
